In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("dataset.csv")


In [3]:
df.columns

Index(['name', 'description', 'make', 'model', 'year', 'price', 'engine',
       'cylinders', 'fuel', 'mileage', 'transmission', 'trim', 'body', 'doors',
       'exterior_color', 'interior_color', 'drivetrain'],
      dtype='object')

In [4]:
df.drop(["model","name","description","make","trim","exterior_color","interior_color"],axis=1,inplace=True)


In [5]:
df.columns

Index(['year', 'price', 'engine', 'cylinders', 'fuel', 'mileage',
       'transmission', 'body', 'doors', 'drivetrain'],
      dtype='object')

In [6]:
df.isnull().any(axis=1).sum()


np.int64(166)

In [7]:
df[df.isnull().any(axis=1)]

,year,price,engine,cylinders,fuel,mileage,transmission,body,doors,drivetrain
14,2024,51695.0,c,NaN,Electric,4.0,1-Speed Automatic,SUV,4.0,All-wheel Drive
27,2024,48705.0,24V MPFI DOHC,6.0,Gasoline,NaN,9-Speed Automatic,Passenger Van,4.0,All-wheel Drive
28,2024,52190.0,c,NaN,Electric,6.0,1-Speed Automatic,SUV,4.0,All-wheel Drive
33,2024,49820.0,c,NaN,Electric,13.0,Automatic,SUV,4.0,All-wheel Drive
35,2024,47790.0,c,NaN,Electric,5.0,1-Speed Automatic,SUV,4.0,All-wheel Drive
...,...,...,...,...,...,...,...,...,...,...
941,2024,38201.0,c,NaN,Electric,12.0,Automatic,SUV,4.0,Rear-wheel Drive
943,2024,NaN,16V GDI DOHC Turbo,4.0,Gasoline,697.0,8-Speed Automatic,SUV,4.0,All-wheel Drive
944,2024,41528.0,c,NaN,Electric,13.0,Automatic,SUV,4.0,All-wheel Drive
978,2024,43439.0,c,NaN,Electric,14.0,Automatic,SUV,4.0,All-wheel Drive


In [8]:
df=df.dropna()

In [15]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

# Load dataset
df = pd.read_csv("dataset.csv")

# Drop unnecessary columns
columns_to_drop = ["name", "description", "make", "trim", "exterior_color", "interior_color", "model"]
df = df.drop(columns=columns_to_drop)

# Handle missing values
num_cols = ["price", "cylinders", "mileage", "doors"]
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

cat_cols = ["engine", "fuel", "transmission", "body", "drivetrain"]
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# Feature Engineering: Age of vehicle
df["age"] = 2025 - df["year"]
df = df.drop(columns=["year"])

# Features and target
X = df.drop(columns=["price"])
y = df["price"]

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)],
    remainder="passthrough"
)

# Random Forest Model
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(n_estimators=300, random_state=42))
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model.fit(X_train, y_train)

# Save model
joblib.dump(model, "vehicle_price_model.pkl")

print("✅ Model trained and saved as vehicle_price_model.pkl")


✅ Model trained and saved as vehicle_price_model.pkl
